# Setup

**Note to Mac users:**
* You might get this error: `Unable to revert mtime: /Library/Fonts`
* The solution is to install the *libmagic* brew: `brew install libmagic`
* Reference: [Python MacOS Error: Unable to revert mtime: /Library/Fonts](https://stackoverflow.com/questions/62279920/python-macos-error-unable-to-revert-mtime-library-fonts)

In [ ]:
import ansible_runner
import collectors as cl
import datetime as dt
import dtale
import helpers as hp
import importlib
import ipywidgets as widgets
import jupyterlab_widgets
import os
import pandas as pd
import readline
import run_collectors as rc

from IPython.display import clear_output
from IPython.display import display

# Reload helper modules after changes. Mostly used for development.
importlib.reload(cl)
importlib.reload(hp)
importlib.reload(rc)

# Do not write history to .python_history to protect credentials
readline.write_history_file = lambda *args: None

# Defining these variables now allows the user to update selections later
collector_select = dict()
hostgroup_select = dict()

# Set Pandas display settings
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

# Define several functions that need to be run within Jupyter
def create_collectors_df(collector_select, hostgroup_select):
    '''
    Creates a dataframe of collectors to execute. Each row contains the
    ansible_os, hostgroup, and collector.

    Args:
        collector_select(dict):    A dictionary of selected collectors
        hostgroup_select (dict):   A dictionary of selected hostgroups

    Returns:
        df_collectors (DataFrame): A dataframe of collectors to run
    '''
    
    df_data = dict()
    df_data['ansible_os'] = list()
    df_data['hostgroup'] = list()
    df_data['collector'] = list()

    for key, value in hostgroup_select.items():
        for item in value:
            if item.value == True:
                collectors = [c.description for c in collector_select.get(key) if \
                              c.value == True]
                for c in collectors:
                    df_data['ansible_os'].append(key)
                    df_data['hostgroup'].append(item.description)
                    df_data['collector'].append(c)
    df_collectors = pd.DataFrame.from_dict(df_data)
    return df_collectors

def select_collectors(collector_select, hostgroup_select):
    '''
    Selects the collectors for the selected hostgroups.

    Args:
        collector_select (dict): The collectors the user selected. The first
                                 time this is run, it will be an empty
                                 dictionary. Passing it to the function
                                 allows the user to select additional
                                 hostgroups later without losing their
                                 selected collectors.
        hostgroup_select (dict): The hostgroups the user selected

    Returns:
        collector_select (dict): A dictionary of collectors to select
    '''
    for key, value in hostgroup_select.items():
        for item in value:
            if item.value == True:
                if not collector_select.get(key):
                    available = hp.define_collectors(key)
                    collector_select[key] = [widgets.Checkbox(value=False,
                                                              description=c,
                                                              disabled=False,
                                                              indent=False) for c in available]
    # Delete any hostgroups that do not have available selectors
    to_delete = list()
    for key, value in collector_select.items():
        if not value:
            to_delete.append(key)
    for item in to_delete:
        del collector_select[item]
                    
    # Delete any hostgroups that the user has de-selected
    for key, value in hostgroup_select.items():
        if collector_select.get(key):
            to_delete = True
            for item in hostgroup_select[key]:
                if item.value == True:
                    to_delete = False
            if to_delete:
                del collector_select[key]
    return collector_select

def select_hostgroups(private_data_dir):
    '''
    Selects the collectors for the selected hostgroups.

    Args:
        collector_select (dict): The collectors the user selected. The first
                                 time this is run, it will be an empty
                                 dictionary. Passing it to the function
                                 allows the user to select additional
                                 hostgroups later without losing their
                                 selected collectors.
        hostgroup_select (dict): The hostgroups the user selected
        private_data_dir (str):  The path to the Ansible private data directory

    Returns:
        collector_select (dict): A dictionary of collectors to select
    '''
    # Define and select hostgroups
    groups = hp.ansible_group_hostgroups_by_os(private_data_dir)    
    for key, value in groups.items():
        if not hostgroup_select.get(key):
            hostgroup_select[key] = [widgets.Checkbox(value=False,
                                                      description=h,
                                                      disabled=False,
                                                      indent=False) for h in value]
    return hostgroup_select

## Set Variables

Gather the following variables:

* database name
* path to the Net-Manage repository
* private data directory
* path to store the output

**Note:** The inventory file is statically set to *private_data_dir/inventory/hosts*

## Explanation of Variables

### database name

defaults to the current date, in YYYY-MM-DD format

### path to the Net-Manage repository

defaults to current folder

### Path to the Private Data Directory

This variable requires explanation. It is the folder that *contains* the inventory folder. **Do not store that information in Net-Manage directory, because it could be overwritten the next time you do a "git pull"**

### path to store the output

path to store the database and Excel files

defaults to 'private_data_dir/output'

**note:** The Ansible-runner output, like "artifacts", will be stored in the private data directory

#### Example folder structure

* Net-Manage path: *~/source/repos/InsightSSG/Net-Manage*

* Private data directory: *~/tenant_data/*

* Inventory file: *~/tenant_data/inventory/hosts*

### Special Note for Merakis

If you do not need to connect to any devices with SSH, you can leave the following variables blank:

* inventories
* private_data_dir
* username
* password

In [ ]:
api_key, db, db_path, inventories, nm_path, out_path, private_data_dir = hp.set_vars()
play_path = f'{nm_path}/playbooks'
username, password = hp.get_creds()

# Select and Run Collectors

## Select the hostgroups to run the collectors on

In [ ]:
hostgroup_select = select_hostgroups(private_data_dir)
for key in sorted(hostgroup_select.keys()):
    print(key.upper())
    display(widgets.GridBox(hostgroup_select[key],
                            layout=widgets.Layout(grid_template_columns='repeat(3, 200px)')))
    print('\n')

## Select the collectors to run

In [ ]:
collector_select = select_collectors(collector_select, hostgroup_select)
for key in sorted(collector_select.keys()):
    print(key.upper())
    display(widgets.GridBox(collector_select[key],
                            layout=widgets.Layout(grid_template_columns='repeat(3, 200px)')))
    print('\n')

## Run Collectors

In [ ]:
# Create a dataframe of collectors to run
print('COLLECTORS TO RUN\n')
df_collectors = create_collectors_df(collector_select, hostgroup_select)
display(df_collectors)

# Set the timestamp so it will be consistent for all collectors
ts = dt.datetime.now()
ts = ts.strftime('%Y-%m-%d_%H%M')

# Execute the collectors
for idx, row in df_collectors.iterrows():
    ansible_os = row['ansible_os']
    hostgroup = row['hostgroup']
    collector = row['collector']
    result = rc.collect(collector,
                        nm_path,
                        private_data_dir,
                        ts,
                        ansible_os=ansible_os,
                        username=username,
                        password=password,
                        api_key=api_key,
                        hostgroup=hostgroup,
                        play_path=play_path,
                        db_path=db_path)
    print(f'RESULT: {ansible_os.upper()} {collector.upper()} COLLECTOR\n')
    display(result)

# Run Validators

In [ ]:
tables = hp.get_database_tables(db_path)
supported_tables = hp.define_supported_validation_tables()
for t in tables:
    diff_cols = list()
    if t in supported_tables:
        for key, value in supported_tables[t].items():
            diff_cols.append(key)
            diff_cols.append(value)
        df_diff = hp.validate_table(t, db_path, diff_cols)
        print(t.upper())
        display(df_diff)

In [ ]:
df = results['interface_summary']
d = dtale.show(df, host='localhost')

In [ ]:
d.open_browser()

In [ ]:
display(df)

In [ ]:
dtale.instances()

In [ ]:
d

In [ ]:
d.open_browser()

In [ ]:
dtale.instances()

In [ ]:
d

In [ ]:
dir(dtale.show)

In [ ]:
import { PageConfig } from '@jupyterlab/coreutils';

// Get base URL of current notebook server
let base_url = PageConfig.getBaseUrl()

// Construct URL of our proxied service
let service_url = base_url + 'proxy/' + port;

In [ ]:

d._main_url

In [ ]:
d

## View Collectors

In [ ]:
results = list()
for c in collectors.split(','):
    df = hp.read_table(db_path, c)
    df = df.set_index('timestamp')
    results.append(df)
for item in results:
    display(item)

In [ ]:
collectors.split(',')
test = ['arp_table', 'abc']

In [ ]:
# accordion = widgets.Accordion(children=[widgets.IntSlider(), widgets.Text()], titles=('Slider', 'Text'))
titles = tuple(c for c in collectors.split(','))
accordion = widgets.Accordion(children=widgets.Output(), titles=(titles))

accordion


In [ ]:
db_path, db, collectors

In [ ]:
try:
    collectors = [c.strip() for c in collectors.split(',')]
except:
    pass
try:
    hostgroups = [h.strip() for h in hostgroups.split(',')]
except:
    pass
df_collectors = hp.ansible_create_collectors_df(hostgroups,
                                                collectors)

display(df_collectors)

df_vars = hp.ansible_create_vars_df(hostgroups, private_data_dir)

display(df_vars)

# Select and Run Validators

# The cells below are a scratchpad for development. They will not be in production.

In [ ]:
# I copied this function from the ansible_runner documentation so that it will
# be easy to reference during development. This function is not part of the
# Net-Manage code.

def get_inventory(action, inventories, response_format=None, host=None, playbook_dir=None,
                  vault_ids=None, vault_password_file=None, output_file=None, export=None, **kwargs):
    '''
    Run an ansible-inventory command to get inventory related details.

    :param action: Valid values are one of ``graph``, ``host``, ``list``
                   ``graph`` create inventory graph, ``host`` returns specific host info and works as inventory script and
                   ``list`` output all hosts info and also works as inventory script.
    :param inventories: List of inventory host path.
    :param response_format: The output format for response. Valid values can be one of ``json``, ``yaml``, ``toml``.
                            Default is ``json``. If ``action`` is ``graph`` only allowed value is ``json``.
    :param host: When ``action`` is set to ``host`` this parameter is used to get the host specific information.
    :param playbook_dir: This parameter is used to sets the relative path for the inventory.
    :param vault_ids: The vault identity to use.
    :param vault_password_file: The vault password files to use.
    :param output_file: The file path in which inventory details should be sent to.
    :param export: The boolean value if set represent in a way that is optimized for export,not as an accurate
                   representation of how Ansible has processed it.
    :param runner_mode: The applicable values are ``pexpect`` and ``subprocess``. Default is set to ``subprocess``.
    :param host_cwd: The host current working directory to be mounted within the container (if enabled) and will be
                     the work directory within container.
    :param envvars: Environment variables to be used when running Ansible. Environment variables will also be
                    read from ``env/envvars`` in ``private_data_dir``
    :param passwords: A dictionary containing password prompt patterns and response values used when processing output from
                      Ansible. Passwords will also be read from ``env/passwords`` in ``private_data_dir``.
    :param settings: A dictionary containing settings values for the ``ansible-runner`` runtime environment. These will also
                     be read from ``env/settings`` in ``private_data_dir``.
    :param ssh_key: The ssh private key passed to ``ssh-agent`` as part of the ansible-playbook run.
    :param quiet: Disable all output
    :param json_mode: Store event data in place of stdout on the console and in the stdout file
    :param artifact_dir: The path to the directory where artifacts should live, this defaults to 'artifacts' under the private data dir
    :param project_dir: The path to the playbook content, this defaults to 'project' within the private data dir
    :param rotate_artifacts: Keep at most n artifact directories, disable with a value of 0 which is the default
    :param timeout: The timeout value in seconds that will be passed to either ``pexpect`` of ``subprocess`` invocation
                    (based on ``runner_mode`` selected) while executing command. It the timeout is triggered it will force cancel the
                    execution.
    :param process_isolation: Enable process isolation, using a container engine (e.g. podman).
    :param process_isolation_executable: Process isolation executable or container engine used to isolate execution. (default: podman)
    :param container_image: Container image to use when running an ansible task (default: quay.io/ansible/ansible-runner:devel)
    :param container_volume_mounts: List of bind mounts in the form 'host_dir:/container_dir:labels. (default: None)
    :param container_options: List of container options to pass to execution engine.
    :param container_workdir: The working directory within the container.
    :param fact_cache: A string that will be used as the name for the subdirectory of the fact cache in artifacts directory.
                       This is only used for 'jsonfile' type fact caches.
    :param fact_cache_type: A string of the type of fact cache to use.  Defaults to 'jsonfile'.
    :param private_data_dir: The directory containing all runner metadata needed to invoke the runner
                             module. Output artifacts will also be stored here for later consumption.
    :param ident: The run identifier for this invocation of Runner. Will be used to create and name
                  the artifact directory holding the results of the invocation.
    :param event_handler: An optional callback that will be invoked any time an event is received by Runner itself, return True to keep the event
    :param cancel_callback: An optional callback that can inform runner to cancel (returning True) or not (returning False)
    :param finished_callback: An optional callback that will be invoked at shutdown after process cleanup.
    :param status_handler: An optional callback that will be invoked any time the status changes (e.g...started, running, failed, successful, timeout)
    :param artifacts_handler: An optional callback that will be invoked at the end of the run to deal with the artifacts from the run.
    :param check_job_event_data: Check if job events data is completely generated. If event data is not completely generated and if
                                 value is set to 'True' it will raise 'AnsibleRunnerException' exception,
                                 if set to 'False' it log a debug message and continue execution. Default value is 'False'
    :type action: str
    :type inventories: list
    :type response_format: str
    :type host: str
    :type playbook_dir: str
    :type vault_ids: str
    :type vault_password_file: str
    :type output_file: str
    :type export: bool
    :type runner_mode: str
    :type host_cwd: str
    :type envvars: dict
    :type passwords: dict
    :type settings: dict
    :type private_data_dir: str
    :type project_dir: str
    :type artifact_dir: str
    :type fact_cache_type: str
    :type fact_cache: str
    :type process_isolation: bool
    :type process_isolation_executable: str
    :type container_image: str
    :type container_volume_mounts: list
    :type container_options: list
    :type container_workdir: str
    :type ident: str
    :type rotate_artifacts: int
    :type timeout: int
    :type ssh_key: str
    :type quiet: bool
    :type json_mode: bool
    :type event_handler: Callable
    :type cancel_callback: Callable
    :type finished_callback: Callable
    :type status_handler: Callable
    :type artifacts_handler: Callable
    :type check_job_event_data: bool

    :returns: Returns a tuple of response and error string. In case if ``runner_mode`` is set to ``pexpect`` the error value is
              empty as ``pexpect`` uses same output descriptor for stdout and stderr. If the vaue of ``response_format`` is ``json``
              it returns a python dictionary object.
    '''

    event_callback_handler = kwargs.pop('event_handler', None)
    status_callback_handler = kwargs.pop('status_handler', None)
    artifacts_handler = kwargs.pop('artifacts_handler', None)
    cancel_callback = kwargs.pop('cancel_callback', None)
    finished_callback = kwargs.pop('finished_callback', None)

    rd = InventoryConfig(**kwargs)
    rd.prepare_inventory_command(action=action, inventories=inventories, response_format=response_format, host=host, playbook_dir=playbook_dir,
                                 vault_ids=vault_ids, vault_password_file=vault_password_file, output_file=output_file, export=export)
    r = Runner(rd,
               event_handler=event_callback_handler,
               status_handler=status_callback_handler,
               artifacts_handler=artifacts_handler,
               cancel_callback=cancel_callback,
               finished_callback=finished_callback)
    r.run()
    response = r.stdout.read()
    error = r.stderr.read()
    if response and response_format == 'json':
        response = json.loads(sanitize_json_response(response))
    return response, error

In [ ]:
print(dir(widgets))

In [ ]:
from IPython.display import display
button = widgets.Button(description="Click Me!")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        print("Button clicked.")

button.on_click(on_button_clicked)